In [1]:
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd

# -------------------------------
# Setup & Configuration
# -------------------------------

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key for Gemini
api_key = os.getenv("Gemini_API_Key")
if not api_key:
    raise ValueError("API key not found. Ensure 'Gemini_API_Key' is correctly set in the .env file.")

genai.configure(api_key=api_key)

# Load the wine dataset
nlp_file_path = "../Resources/nlp_3.1_final.csv"
wine_df = pd.read_csv(nlp_file_path)

# -------------------------------
# Define Options for the UI
# -------------------------------

# Wine Color Options with Descriptions
wine_colors = {
    "Red": "Ranges from light (garnet) to deep (opaque) reds. Full-bodied reds tend to have higher tannins.",
    "White": "Ranges from clear to golden hues, with fuller-bodied whites having a richer color due to oak aging.",
    "Rosé": "Varies from pale salmon to magenta, depending on grape variety and skin contact time.",
    "Orange": "A white wine made with extended skin contact, giving it a deep amber color.",
    "Sparkling": "Can be white, rosé, or even red with effervescence from natural carbonation."
}

# Flavor options
flavor_options = [
    "Citrus", "Berries", "Stone Fruits", "Tropical Fruits", "Dried Fruits",
    "Floral", "Herbal", "Vegetal", "Spices", "Woody", "Nutty",
    "Toasty", "Earthy", "Mineral", "Chocolate & Coffee", "Caramel & Vanilla",
    "Sweet", "Savory", "Oaky", "Smoky"
]

# Wine body options
body_options = ["Light-bodied", "Medium-bodied", "Full-bodied"]

# Aroma Options (Category - Subcategory format)
aroma_options = [
    "Fruity White Wine - Citrus",
    "Fruity White Wine - White Berries",
    "Fruity White Wine - Pome Fruits",
    "Fruity White Wine - Stone Fruits",
    "Fruity White Wine - Tropical Fruits",
    "Fruity White Wine - Botrytized",
    "Fruity Red Wine - Tropical Fruits",
    "Fruity Red Wine - Red Berries",
    "Fruity Red Wine - Stone Fruits",
    "Fruity Red Wine - Fortified",
    "Floral - White Flowers",
    "Floral - Colored Flowers",
    "Vegetal - Vegetables",
    "Vegetal - Fresh Herbs",
    "Vegetal - Dried Herbs",
    "Vegetal - Leaves",
    "Maturation in Oak Barrel - Woods",
    "Maturation in Oak Barrel - Nuts",
    "Maturation in Oak Barrel - Spices",
    "Maturation in Oak Barrel - Toasted"
]

# Price Selection (Dropdown with explicit ranges)
price_options = {
    "$1 - $9.99": "Budget-friendly wines, great for casual drinking.",
    "$10 - $99.99": "Standard quality wines, suitable for most occasions.",
    "$100 - $999.99": "Premium and high-quality wines for special occasions.",
    "$1000+": "Rare, collectible, and investment-grade wines."
}

# -------------------------------
# Core Functions
# -------------------------------

def get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range):
    """
    Searches the dataset for a wine recommendation based on user inputs.
    If no match is found, uses Gemini AI to generate a recommendation.
    Returns a formatted recommendation string and the wine name.
    """
    filtered_wines = wine_df[
        wine_df["color"].apply(lambda x: wine_color in x) &
        wine_df["flavors"].apply(lambda x: flavor_profile in x) &
        wine_df["aromas"].apply(lambda x: aroma in x)
    ]
    
    if not filtered_wines.empty:
        recommended_wine = filtered_wines.sample(1).iloc[0]
        wine_name = recommended_wine['Name']
        wine_recommendation = f"""
**Recommended Wine:**  
**{wine_name}**

**Attributes:**  
- **Color:** {recommended_wine['color']}  
- **Flavor Profile:** {flavor_profile}  
- **Aroma:** {recommended_wine['aromas']}  
- **Sweetness Level:** {sweetness}  
- **Body:** {body}  
- **Price:** {price_range}  
"""
    else:
        prompt = f"""
You are a wine expert. Based on the following attributes, recommend a wine:
- Wine Color: {wine_color}
- Flavor Profile: {flavor_profile}
- Aroma: {aroma}
- Sweetness Level: {sweetness}
- Body: {body}
- Price Range: {price_range}
Provide the best wine recommendation with its name and key details.
"""
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        wine_recommendation = response.text
        wine_name = (response.text.split('\n')[2].strip('**')
                     if '\n' in response.text and len(response.text.split('\n')) > 2
                     else wine_color)
    
    return wine_recommendation, wine_name

def get_food_pairing(wine_name):
    """
    Uses Gemini AI to generate a one-sentence food pairing suggestion.
    """
    prompt = f"Given the wine '{wine_name}', suggest a food pairing that would best complement it. Provide the answer in one sentence."
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    food_pairing = response.text.strip()
    return food_pairing

def get_recipe_and_shopping_list(food_pairing):
    """
    Uses Gemini AI to provide a detailed recipe and a bullet‑formatted shopping list
    for a dish that pairs well with the given food pairing suggestion.
    """
    prompt = f"""
Provide a detailed recipe for preparing a dish that pairs well with {food_pairing}. 
Include step-by-step cooking instructions, a full list of ingredients, and a separate shopping list formatted as bullet points.
"""
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    full_recipe = response.text.strip()
    
    # Extract shopping list items by searching for lines starting with a dash.
    shopping_list = []
    for line in full_recipe.splitlines():
        if line.strip().startswith("-"):
            shopping_list.append(line.strip())
    
    return full_recipe, shopping_list

def get_full_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range, view_mode):
    """
    Combines the wine recommendation, food pairing, recipe, and shopping list into one formatted output.
    The output is formatted with collapsible sections using <details> tags.
    The 'view_mode' parameter controls whether a Detailed or Summary view is returned.
    Returns the full recommendation text and the wine name.
    """
    wine_rec, wine_name = get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range)
    food_pairing = get_food_pairing(wine_name)
    recipe, shopping_list = get_recipe_and_shopping_list(food_pairing)
    
    # Create a clickable link for the wine name (opens a Google search)
    wine_link = f"[**{wine_name}**](https://www.google.com/search?q={wine_name.replace(' ', '+')})"
    
    if view_mode == "Detailed":
        full_output = f"""
# Wine & Food Pairing Recommendation

**Wine Recommendation:**  
{wine_link}  
{wine_rec}

<details>
  <summary><strong>Food Pairing Suggestion</strong></summary>
  {food_pairing}
</details>

<details>
  <summary><strong>Recipe & Instructions</strong></summary>
  {recipe}
</details>

<details>
  <summary><strong>Shopping/Ingredient List</strong></summary>
  {''.join(f"- {item}\n" for item in shopping_list)}
</details>
"""
    else:
        full_output = f"""
# Wine & Food Pairing Recommendation (Summary)

**Wine Recommendation:**  
{wine_link}  
{wine_rec}

**Food Pairing:** {food_pairing}

For full recipe details and the shopping list, please download the detailed recommendation.
"""
    return full_output, wine_name

def open_google_search(wine_name):
    """
    Instead of opening the URL on the server, returns a clickable HTML link.
    This link will open in the user's browser when clicked.
    """
    search_url = f"https://www.google.com/search?q={wine_name.replace(' ', '+')}"
    return f'<a href="{search_url}" target="_blank">Click here to search for {wine_name} on Google</a>'

def save_recommendation(full_text):
    """
    Saves the full recommendation text to a file for downloading.
    Returns the file path.
    """
    file_path = "wine_recommendation.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(full_text)
    return file_path

def get_printable_recommendation(full_text):
    """
    Returns an HTML page containing the recommendation along with a button that triggers the browser’s print dialog.
    """
    printable_html = f"""
<html>
  <head>
    <title>Printable Wine Recommendation</title>
    <style>
      body {{ font-family: Arial, sans-serif; margin: 40px; }}
      .container {{ max-width: 800px; margin: auto; }}
      h1 {{ text-align: center; }}
      details {{ margin-bottom: 20px; }}
    </style>
  </head>
  <body>
    <div class="container">
      {full_text}
      <br><br>
      <button onclick="window.print()">Print this page</button>
    </div>
  </body>
</html>
"""
    return printable_html

def submit_feedback(feedback, comments):
    """
    Processes the user's feedback. In this example, the feedback is simply printed to the console.
    You could extend this to log the feedback to a file or database.
    """
    print("User Feedback:", feedback, comments)
    return "Thank you for your feedback!"

# -------------------------------
# Gradio Interface
# -------------------------------

with gr.Blocks() as demo:
    # State to hold the wine name (used by the Google search button)
    wine_name_state = gr.State(value="")
    
    gr.Markdown("# Wine & Food Pairing Recommendation System")
    gr.Markdown("Select your preferences and view your personalized recommendation. Use the tabs below to switch between the recommendation, a printer-friendly version, and to provide feedback.")
    
    with gr.Row():
        wine_color_in = gr.Dropdown(list(wine_colors.keys()), label="Wine Color")
        flavor_profile_in = gr.Dropdown(flavor_options, label="Flavor Profile")
        aroma_in = gr.Dropdown(aroma_options, label="Aroma Preference")
    with gr.Row():
        sweetness_in = gr.Slider(0, 10, step=1, label="Sweetness Level (0 = Dry, 10 = Very Sweet)")
        body_in = gr.Dropdown(body_options, label="Wine Body")
        price_range_in = gr.Dropdown(list(price_options.keys()), label="Price Range")
    
    view_mode_in = gr.Radio(["Detailed", "Summary"], label="View Mode", value="Detailed")
    
    with gr.Tabs():
        with gr.TabItem("Recommendation"):
            recommendation_output = gr.Markdown(label="Full Recommendation", elem_id="recommendation-markdown")
            with gr.Row():
                submit_btn = gr.Button("Get Full Recommendation", variant="primary")
                search_button = gr.Button("Find My Wine", variant="secondary")
            # Add an output component for the clickable search link.
            search_link_output = gr.HTML(label="Google Search Link")
        
        with gr.TabItem("Printable Version"):
            printable_output = gr.HTML(label="Printable Recommendation")
            print_btn = gr.Button("Generate Printable Version", variant="secondary")
        
        with gr.TabItem("Download & Share"):
            download_btn = gr.Button("Download Recommendation", variant="secondary")
            download_file = gr.File(label="Download Your Recommendation")
        
        with gr.TabItem("Feedback"):
            feedback_radio = gr.Radio(["Yes", "No"], label="Was this recommendation helpful?")
            feedback_comments = gr.Textbox(label="Additional Comments (optional)", lines=3)
            submit_feedback_btn = gr.Button("Submit Feedback", variant="primary")
            feedback_response = gr.Markdown()
    
    # -------------------------------
    # Connect UI Actions
    # -------------------------------
    
    # Generate the full recommendation and update the hidden wine name state
    submit_btn.click(
        fn=get_full_recommendation,
        inputs=[wine_color_in, flavor_profile_in, aroma_in, sweetness_in, body_in, price_range_in, view_mode_in],
        outputs=[recommendation_output, wine_name_state]
    )
    
    # Return a clickable Google search link for the wine name when "Find My Wine" is clicked
    search_button.click(
        fn=open_google_search,
        inputs=[wine_name_state],
        outputs=[search_link_output]
    )
    
    # Download the recommendation as a text file
    download_btn.click(
        fn=save_recommendation,
        inputs=[recommendation_output],
        outputs=download_file
    )
    
    # Generate a printer-friendly HTML version with a print button
    print_btn.click(
        fn=lambda rec: get_printable_recommendation(rec),
        inputs=[recommendation_output],
        outputs=printable_output
    )
    
    # Process user feedback
    submit_feedback_btn.click(
        fn=submit_feedback,
        inputs=[feedback_radio, feedback_comments],
        outputs=feedback_response
    )
    
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: c:\Users\Johnathan\anaconda3\envs\dev1\Lib\site-packages\gradio
